In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
from datetime import datetime
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
links = [
"alam-semulajadi",
"astronomi-kosmologi",
"berita-peristiwa",
"bicara-saintis/suara-saintis-muda/",
"fiksyen-buku-filem",
"fizik",
"kimia",
"komputer-it",
"luar-negara"
,"matematik",
"perubatan-kesihatan",
"rencana",
"sejarah-falsafah",
"teknologi-kejuruteraan",
"tempatan",
"tenaga",
"tokoh"
]

hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 



In [3]:
for i in tqdm(links):
    
    x = 1

    while True:

        while True:
            try:
                r = requests.get(f'https://www.majalahsains.com/category/{i}/page/{x}',headers = headers)
                break
            except Exception as e:
                print(e)
                time.sleep(1.0)
                
        soup = BeautifulSoup(r.text, "lxml")
      
        if soup.find('h1', attrs = {"class":"jeg_archive_title"}) is not None:
            break

        for link in soup.find_all('h3', attrs = {"class":"jeg_post_title"}):
            
            href = link.find('a').get('href')
            
           
            hrefs.append(href)

        
        x = x + 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [09:32<00:00, 33.65s/it]


In [4]:
hrefs = list(set(hrefs))

In [7]:
with open('majalahsains-link.json', 'a') as f:
    json.dump(hrefs, f)

In [6]:
len(hrefs)

1540

In [3]:
with open('majalahsains-link.json') as fopen:
    hrefs = json.load(fopen)

In [4]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', attrs = {"class":"jeg_post_title"}).text
        h = soup.find('div', attrs = {"class":"content-inner"})        
        p = [p.text for p in h.find_all() if len(p.text.split()) > 20]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None 
    
    tag = soup.find('div', class_="jeg_post_tags")
    
    if tag:
        tags = [tags.text for tags in tag.find_all("a")]
    else:
        tags = None
    
    
    
    data = {'url': x, 'headline': headline, 'content': p, 'tags': tags}
    return data

In [5]:
max_workers = 30

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('majalahsains-2.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1540/1540 [35:03<00:00,  1.37s/it]
